In [1]:
import numpy as np
from models.dae1d import DiamondDAE1D

from sklearn.model_selection import GroupKFold

In [2]:
import os

os.environ['KERAS_BACKEND'] = 'torch'

import keras

In [3]:
clean = np.load("data/processed/clean.npy")
noisy = np.load("data/processed/noisy.npy")

In [4]:
points = np.repeat(np.arange(1200), 300)
points.shape


(360000,)

In [5]:
clean = np.reshape(clean, (-1, 768, 1))
noisy = np.reshape(noisy, (-1, 768, 1))

In [6]:
kfold = GroupKFold(n_splits=5)

In [7]:
train, validation = next(kfold.split(noisy.squeeze(), clean.squeeze(), points))

In [8]:
dae = DiamondDAE1D(
    n_convlayers=10,
    filters=[32, 32, 64, 64, 128],
    kernel_size=11,
)

In [9]:
dae.build_model((768, 1))

In [10]:
dae.summary()

Model: "DiamondDAE1D"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 768, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_0 (Conv1D)         │ (None, 768, 32)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_pool_0 (MaxPooling1D)   │ (None, 384, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_1 (Conv1D)         │ (None, 384, 32)        │        11,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_pool_1 (MaxPooling1D)   │ (None, 192, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_2 (Conv1D)         │ (None, 192, 64)        │        22,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_pool_2 (MaxPooling1D)   │ (None, 96, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_3 (Conv1D)         │ (None, 96, 64)         │        45,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_pool_3 (MaxPooling1D)   │ (None, 48, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_4 (Conv1D)         │ (None, 48, 128)        │        90,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_pool_4 (MaxPooling1D)   │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_upsample_0              │ (None, 48, 128)        │             0 │
│ (UpSampling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_conv_0                  │ (None, 48, 128)        │       180,352 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_upsample_1              │ (None, 96, 128)        │             0 │
│ (UpSampling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_conv_1                  │ (None, 96, 64)         │        90,176 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_upsample_2              │ (None, 192, 64)        │             0 │
│ (UpSampling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_conv_2                  │ (None, 192, 64)        │        45,120 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_upsample_3              │ (None, 384, 64)        │             0 │
│ (UpSampling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_conv_3                  │ (None, 384, 32)        │        22,560 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_upsample_4              │ (None, 768, 32)        │             0 │
│ (UpSampling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 519,489 (1.98 MB)

 Trainable params: 519,489 (1.98 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
dae.train_model(
    noisy[train], clean[train],
    x_val=noisy[validation], y_val=clean[validation],
    batch_size=128, epochs=5
)

Epoch 1/5
1917/9000 ━━━━━━━━━━━━━━━━━━━━ 1:18 11ms/step - loss: 9.8981e-05

KeyboardInterrupt: 